In [8]:
import numpy as np
import chess
import time
import pickle
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
clf = MLPClassifier(hidden_layer_sizes=(1000), alpha=0.0001, max_iter=1,
                    solver='adam', warm_start=True, shuffle=False, verbose=True, early_stopping=True)

def load_model(filename):
    clf = pickle.load(open(filename, 'rb'))
    return clf

def save_model(filename):
    pickle.dump(clf, open(filename, 'wb'))
    
def get_pieces(board):
    i = 0
    pieces = [[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1)], [(-2, -1), (-2, -1)], [(-3, -1), (-3, -1)], [(-4, -1), (-4, -1)], [(-5, -1)], [(-6, -1)],
              [(1, -1), (1, -1), (1, -1), (1, -1), (1, -1), (1, -1), (1, -1), (1, -1)], [(2, -1), (2, -1)], [(3, -1), (3, -1)], [(4, -1), (4, -1)], [(5, -1)], [(6, -1)]]
    stupid_game_mechanic = []
    while i < 64:
        if board.piece_at(i):
            f = 1
            c = 0 if (board.piece_at(i).color) else 1
            for j, piece in enumerate(pieces[board.piece_type_at(i)+(6*c)-1]):
                if piece[1] == -1:
                    pieces[board.piece_type_at(i)+(6*c)-1][j] = (piece[0], i)
                    f = 0
                    break
            if f:
                a = (piece[0], i)
                stupid_game_mechanic.append(a) 
        i+=1

    while 16-len(stupid_game_mechanic):
        a = (1,-1)
        stupid_game_mechanic.append(a)
    ret = []
    for i in pieces:
        ret +=i

    return ret+stupid_game_mechanic

def attacked_squares(board, color):
    attacked = chess.SquareSet()
    for attacker in chess.SquareSet(board.occupied_co[color]):
        attacked |= board.attacks(attacker)
    return attacked

def atk_lst(board):
    w = attacked_squares(board, chess.WHITE)
    b = attacked_squares(board, chess.BLACK)
    lst = []
    for i in range(64):
        if w & (1 << i):
            white_attacks = 1
        else:
            white_attacks = 0
        
        if b & (1 << i):
            black_attacks = 1
        else:
            black_attacks = 0
        
        lst.append((white_attacks, black_attacks))
    
    return lst

def get_best_move(board):
    scores = []
    moves = list(board.legal_moves)
    for move in moves:
        board.push(move)
        opt = np.array(board_eval(board)).reshape(1, -1)
        probs = clf.predict_proba(opt)
        2 if board.turn == chess.BLACK else 0
        scores.append(abs(probs[0][2 if board.turn == chess.BLACK else 0]))
        board.pop()
    return moves[np.argmax(scores)]

#test fen "4k2r/6r1/8/8/8/8/3R4/R3K3 w Qk - 0 1"
b = chess.Board("4k2r/6r1/8/8/8/8/3R4/R3K3 w Qk - 6 9")

def board_eval(b): 
    data = get_pieces(b) + atk_lst(b) + [(b.halfmove_clock, b.fullmove_number)]
    return np.array(data).flatten()



In [2]:
import random

class ChessBot:
    def __init__(self, model, exploration_rate=0.1):
        self.model = model
        self.moves = []
        self.exploration_rate = exploration_rate
        
    def move(self, board):
        legal_moves = list(board.legal_moves)
        
        if random.random() < self.exploration_rate:
            move = random.choice(legal_moves)
        else:
            try:
                move = get_best_move(board)
            except:
                move = random.choice(legal_moves)
        
        self.moves.append(move)
        
        return move

In [3]:
from IPython.display import clear_output

def play_game_and_learn(model, exploration_rate_black=0.1, exploration_rate_white=0.1, should_visualise=False):
	black = ChessBot(model, exploration_rate_black)
	white = ChessBot(model, exploration_rate_white)
	game_positions_data = []
	board = chess.Board()

	if should_visualise:
		display(board)

	while not board.is_game_over():
		board.push(black.move(board) if board.turn == chess.BLACK else white.move(board))
		game_positions_data.append(board_eval(board))

		if should_visualise:
			clear_output(wait=True)
			display(board)

	# TODO: Train the model on the results of the game
	return board.result(), game_positions_data

In [4]:

def run_epoch(rate_b, rate_w, num_games):
	results = []
	data = []
	labels = []
	for _ in range(num_games):
		result, board_states = play_game_and_learn(None, rate_b, rate_w)
		if result == '1-0':
			labels += [1]*len(board_states)
		elif result == '0-1':
			labels += [-1]*len(board_states)
		else:
			labels += [0]*len(board_states)
		
		data += board_states
		results.append(result)
	for i in ['1-0', '0-1', '1/2-1/2']:
		print(f'{i}: {results.count(i)}')

	rate_b = rate_b + results.count('1-0')/len(results)
	rate_w = rate_w + results.count('0-1')/len(results)
	return data, labels, rate_b, rate_w

def train_epoch(labels, data):
	X_train, _, y_train, _ = train_test_split(data, labels ,shuffle=False, test_size=1)
	clf.fit(X_train, y_train)



In [5]:
def train_n_epochs(epochs, rate, decay):
    i = 0
    rate_b = rate
    rate_w = rate
    while i<epochs:
        data, labels, rate_b, rate_w = run_epoch(rate_b*decay, rate_w*decay, 1000)
        train_epoch(labels, data)
        decay *= decay
        i+=1
    b = chess.Board()
    while not b.is_game_over():
        clear_output(wait=True)
        b.push(get_best_move(b))
        display(b)
        time.sleep(0.5)
